<a href="https://colab.research.google.com/github/philberns/FTRN65/blob/main/ex3_cosinus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


============================
Underfitting vs. Overfitting
============================

This example demonstrates the problems of underfitting and overfitting and
how we can use linear regression with polynomial features to approximate
nonlinear functions. The plot shows the function that we want to approximate,
which is a part of the cosine function. In addition, the samples from the
real function and the approximations of different models are displayed. The
models have polynomial features of different degrees. We can see that a
linear function (polynomial with degree 1) is not sufficient to fit the
training samples. This is called **underfitting**. A polynomial of degree 4
approximates the true function almost perfectly. However, for higher degrees
the model will **overfit** the training data, i.e. it learns the noise of the
training data.
We evaluate quantitatively **overfitting** / **underfitting** by using
cross-validation. We calculate the mean squared error (MSE) on the validation
set, the higher, the less likely the model generalizes correctly from the
training data.


In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

def true_fun(X):
    return np.cos(1.5 * np.pi * X)

np.random.seed(20)
n_samples = 25
max_deg = 10
degrees = list(range(1,max_deg+1))
sig = 0.1; # noise level

X = np.sort(np.random.rand(n_samples))
y = true_fun(X) + np.random.randn(n_samples) * 0.1
t = np.zeros(max_deg)
cv = np.zeros(max_deg)
for i in range(len(degrees)):
    polynomial_features = PolynomialFeatures(degree=degrees[i],
                                             include_bias=True)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X[:, np.newaxis], y)

    # Evaluate the models using crossvalidation
    scores = cross_val_score(pipeline, X[:, np.newaxis], y,
                             scoring="neg_mean_squared_error", cv=25)
    X_test = np.linspace(0, 1, 100)
    y1 = pipeline.predict(X_test[:, np.newaxis])
    y2 = true_fun(X_test)
    t[i]= np.mean(np.square(y1-y2)) + sig**2
    cv[i] = -scores.mean()
    #print(scores)
    plt.plot(X_test, y1 , label="Model")
    plt.plot(X_test, y2, label="True function")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}".format(
        degrees[i], -scores.mean()))
    fig = plt.gcf()
    plt.show()
    #fig.savefig('plot' + str(i+1) +  'moredata.eps',format='eps')


In [ ]:
plt.plot(degrees,t,label="true MSE")
plt.ylim((0, 0.2))
plt.legend(loc="upper center")
plt.xlabel("Order")
plt.ylabel("Mean Square Error")
fig2 = plt.gcf()
plt.show()
#fig2.savefig("MSEvsOrdermoredata.eps", format='eps')

In [ ]:
plt.plot(degrees,t,label="true MSE")
plt.plot(degrees,cv,label="cross validation MSE")
plt.ylim((0, 0.2))
plt.legend(loc="best")
plt.xlabel("Order")
plt.ylabel("Mean Square Error")
fig3 = plt.gcf()
plt.show()
#fig3.savefig("CVmoredata.eps", format = 'eps', dpi=300)